In [1]:
!pip install psycopg2-binary sqlalchemy pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 2.6 MB/s eta 0:00:00


In [2]:
import pandas as pd
import os
from sqlalchemy import create_engine

# Load database credentials from environment variables (with default fallbacks)
db_user = os.getenv("PG_USER", "postgres")
db_pass = os.getenv("PG_PASSWORD", "isba_4715")
db_host = os.getenv("PG_HOST", "isba-dev-02.ctwgc6gcobls.us-east-2.rds.amazonaws.com")
db_port = os.getenv("PG_PORT", "5432")
db_name = os.getenv("PG_DB", "sql_project")

# Create PostgreSQL connection using SQLAlchemy
engine = create_engine(f'postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}')

# Display all rows in results
pd.set_option('display.max_rows', None)

Descriptive Analytics Business Question: Which streaming platforms have the highest total viewing time across the top-ranked shows?

In [3]:
sql_query = '''
SELECT
    "Platform",
    SUM("Minutes Viewed") AS total_minutes_viewed
FROM nielsen_streaming_rankings
GROUP BY "Platform"
ORDER BY total_minutes_viewed DESC
'''
df1 = pd.read_sql(sql_query, engine)
df1


,Platform,total_minutes_viewed
0,Netflix,3562.0
1,Disney+,1718.0
2,Prime Video,1303.0
3,Max,1106.0
4,Hulu / Netflix,930.0
5,Hulu / Netflix / Paramount+,855.0
6,Paramount+,776.0


Insight:

Netflix dominates total minutes viewed with 3,562 minutes, more than 2x the second-place Disney+. Shared-platform content (e.g. Hulu / Netflix / Paramount+) ranks lower in total viewership, suggesting less concentrated traffic.

Recommendation:

Streaming services should prioritize exclusive releases to capture focused engagement. Netflix should continue leveraging original content dominance, while platforms like Paramount+ could benefit from clearer branding and exclusivity.

Prediction:

If current trends persist, Netflix will remain the market leader in streaming time. However, Disney+ and Prime Video are likely to narrow the gap if they continue investing in high-performing titles and exclusive content rights.

Diagnostic Business Question: Do certain platforms rely on one breakout show for their high total minutes viewed, or do they distribute engagement across multiple shows?





In [9]:
sql_query = '''
SELECT
    "Platform",
    COUNT(*) AS show_count,
    MAX("Minutes Viewed") AS top_show_minutes,
    SUM("Minutes Viewed") AS total_minutes,
    ROUND(
        MAX("Minutes Viewed")::numeric / SUM("Minutes Viewed"), 2
    ) AS top_show_share
FROM nielsen_streaming_rankings
GROUP BY "Platform"
ORDER BY top_show_share DESC;
'''

df2 = pd.read_sql(sql_query, engine)
df2


,Platform,show_count,top_show_minutes,total_minutes,top_show_share
0,Prime Video,1,1303,1303.0,1.00
1,Hulu / Netflix,1,930,930.0,1.00
2,Max,1,1106,1106.0,1.00
3,Hulu / Netflix / Paramount+,1,855,855.0,1.00
4,Paramount+,1,776,776.0,1.00
5,Disney+,2,968,1718.0,0.56
6,Netflix,3,1839,3562.0,0.52


Insight:

Most platforms in the dataset rely entirely on a single show for all their streaming engagement (top_show_share = 1.00). In contrast, Netflix (52%) and Disney+ (56%) have broader engagement spread across multiple programs. This suggests they have a more diverse and sticky content library, while others may depend on one standout title to drive viewership.

Recommendation:

For platforms with 100% top-show reliance, it's a business risk to have all engagement tied to a single asset. These platforms should invest in developing or acquiring supporting content to sustain user interest beyond a breakout title. Netflix and Disney+ serve as models of diversified viewership distribution.

Prediction:

If competing platforms fail to expand their content engagement beyond one title, their total minutes viewed may drop sharply once that flagship show declines or rotates out. Platforms with balanced engagement, like Netflix, will likely retain more stable and loyal viewership over time.